<a href="https://colab.research.google.com/github/VishalSharma99/Neural-Cipher-Generator/blob/main/Neural_Cipher_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries if not already available
!pip install tensorflow numpy matplotlib

# Import the libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import random
import string
